In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV,train_test_split,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_curve,auc
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [2]:
def one_stat_result(model, x_train, y_train, x_test, y_test,b_seuil_label=False):
    train_predictions = model.predict(x_train.to_numpy())
    test_predictions = model.predict(x_test.to_numpy())
    if b_seuil_label==True:
        train_predictions = (train_predictions>=0.5).astype(int)
        test_predictions = (test_predictions>=0.5).astype(int)
    print("test_predictions:")
    print(test_predictions)
    cm = confusion_matrix(y_test, test_predictions)
    print(cm)
    print(">> resultat de la classification:")
    print(classification_report(y_test, test_predictions))
    TP = cm[0,0]
    TN = cm[1,1]
    FP = cm[0,1]
    FN = cm[1,0]
    classification_accuracy = (TP + TN) / float(TP + TN + FP + FN)
    print('Justesse de la Classification (accuracy) : {0:0.4f}'.format(classification_accuracy))
    classification_error = (FP + FN) / float(TP + TN + FP + FN)
    print('Erreurs de Classification : {0:0.4f}'.format(classification_error))
    recall = TP / float(TP + FN)
    print('Recall ou Sensitivity : {0:0.4f}'.format(recall))
    print("taux de faux positive:")
    false_positive_rate = FP / float(FP + TN)
    print(false_positive_rate)
    print('False Positive Rate : {0:0.4f}'.format(false_positive_rate))
    specificity = TN / (TN + FP)
    print('Specificity : {0:0.4f}'.format(specificity))

In [3]:
path = r'C:\Users\karl\Documents\datasets\playground23'
df_train = pd.read_csv("df_train_new.csv",sep=";")
df_train = df_train.drop(["Unnamed: 0","id"],axis=1)
df_test = pd.read_csv("df_test_new.csv",sep=";")
df_test = df_test.drop(["Unnamed: 0","id"],axis=1)
df_test

,sudden_fever,headache,mouth_bleed,nose_bleed,muscle_pain,joint_pain,vomiting,rash,diarrhea,hypotension,...,stiff_neck,irritability,confusion,tremor,paralysis,lymph_swells,lips_irritation,toenail_loss,speech_problem,bullseye_rash
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
299,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
301,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_train["prognosis"].value_counts(ascending=True)

Malaria                  48
Lyme_disease             52
Plague                   53
Zika                     58
Yellow_Fever             61
Dengue                   63
Chikungunya              66
Tungiasis                70
Rift_Valley_fever        70
Japanese_encephalitis    81
West_Nile_fever          85
Name: prognosis, dtype: int64

In [5]:
X = df_train.drop(["prognosis"],axis=1)
y = df_train['prognosis']
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=123,stratify=y)

In [95]:
X = df_train.drop(["prognosis"],axis=1)
y = df_train['prognosis']
x_train_,x_test_,y_train_,y_test_ = train_test_split(X,y,test_size=0.2,random_state=123,stratify=y)

In [6]:

# Créer une instance de LabelEncoder
label_encoder = LabelEncoder()

# Convertir les classes de chaînes de caractères en entiers
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Afficher les classes encodées
print("Classes encodées : ", y_train)

Classes encodées :  [ 0  7 10  2  2  7  0  0  8  2  8  1  4  2  5  4  5  6  2  2  4  5 10  3
  2  5  2  7  1 10  6 10  7  1  7  2  4  5  0  8  7  8 10  4  9  8  7  5
  2 10  8  2  6  5  1  7  1  9  6  5  0  4  1  3  7  3  4  1  8  2  4  2
  8  3  7  6  5  0  4  6 10  5  1  3  3  0  7  3 10  9  9  9  9  1 10  6
  1  6  8  9  1  0  8  0  7  6  2  6  4  8 10  7  7  1 10  0  3  8  8  5
  1 10  8  1  5  9  9  8 10  8  4  9  3  2  8  1  9 10  6  2  3  2  2 10
  5  8 10  6  7  1  4  0  6  3  0  9  5  2  1  3  9  1  0  5  7  7  6  9
  2  7 10  3  2  7  3  6  9  6  1  4  7  8  7  9  7  7  2  1  2  2  7  6
  7  5  7  2  9  8  2  3  0  6  5  1  8  4  0  0  4  3  7  3 10  4  6  6
  1 10  8  8  6  3  7  0  7  5 10  4  7  3  8  2  8  8  2  8  6  0  6  7
  6  2  2  4  8  3  3  3  2  2  0  8  9  6  0  6  5  5  8  5  2  8  3  7
  9  8  0  2 10  9  5  6  8  7  4  2  8 10  0  9  9  8  4  8  2  9  8  3
  0  4  9  2  2  1  3  4  6 10  0  2  3  2  6  2  2  9 10  7  4  6  0  9
  8  0  2  1 10  7  1  3 10  7 

In [7]:
x_train_1 = x_train.copy()
y_train_1 = y_train.copy()
x_test_1 = x_test.copy()
y_test_1 = y_test.copy()

In [8]:
ovr = OneVsRestClassifier(LogisticRegression(solver="saga",penalty='l1',C=0.1))
ovr.fit(x_train_1,y_train_1)
one_stat_result(ovr, x_train_1, y_train_1, x_test_1, y_test_1,b_seuil_label=False)

test_predictions:
[8 3 3 2 2 7 2 2 2 0 0 8 7 0 2 0 3 3 7 8 1 8 3 2 2 8 0 8 3 0 3 2 8 2 2 7 0
 3 1 8 9 3 0 8 3 3 0 2 3 7 8 2 2 3 7 7 2 8 2 8 0 7 8 0 3 8 7 8 2 8 7 2 8 7
 8 3 2 7 1 0 3 2 3 0 3 8 0 3 3 8 0 8 3 1 7 0 8 0 3 8 8 8 3 2 2 8 2 3 7 0 2
 7 7 8 7 3 2 8 2 3 0 7 2 8 3 8 8 7 0 3 0 0 3 0 0 2 3 9 8 3 8 2]
[[10  0  2  0  0  0  0  0  1  0  0]
 [ 2  0  1  3  0  0  0  3  4  0  0]
 [ 4  1  1  5  0  0  0  1  3  1  0]
 [ 0  0  0  8  0  0  0  1  1  0  0]
 [ 1  0  1  4  0  0  0  0  4  0  0]
 [ 0  0  4  4  0  0  0  1  2  0  0]
 [ 3  1  3  1  0  0  0  1  5  0  0]
 [ 0  2  3  0  0  0  0  9  0  0  0]
 [ 2  0  2  4  0  0  0  3  6  0  0]
 [ 2  0  8  0  0  0  0  0  2  0  0]
 [ 0  0  4  2  0  0  0  0  5  1  0]]
>> resultat de la classification:
              precision    recall  f1-score   support

           0       0.42      0.77      0.54        13
           1       0.00      0.00      0.00        13
           2       0.03      0.06      0.04        16
           3       0.26      0.80      0.39  

C:\Users\karl\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\karl\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\karl\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\karl\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
C:\Users\karl\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but Logist

In [9]:
x_train_2 = x_train.copy()
y_train_2 = y_train.copy()
x_test_2 = x_test.copy()
y_test_2 = y_test.copy()

In [10]:
nb_label = len(set(list(y_train_2)))
nb_label

11

In [11]:
param_grid = {
    'max_depth': list(range(2,6,1)),
    'learning_rate': [0.2,0.1,0.01,0.001],
    'n_estimators': [50,100,300,400],
    'reg_lambda': [0.5,0.1],
    'objective': ['multi:softmax'],
    'num_class': [nb_label], # Remplacez num_classes par le nombre de classes dans votre ensemble de données
}

model = xgb.XGBClassifier()
grid_search = GridSearchCV(model, param_grid, cv=10, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train_2, y_train_2)
grid_search.best_params_

{'learning_rate': 0.1,
 'max_depth': 3,
 'n_estimators': 100,
 'num_class': 11,
 'objective': 'multi:softmax',
 'reg_lambda': 0.5}

In [12]:
def one_stat_result(model, x_train, y_train, x_test, y_test,b_seuil_label=False):
    train_predictions = model.predict(x_train)
    test_predictions = model.predict(x_test)
    if b_seuil_label==True:
        train_predictions = (train_predictions>=0.5).astype(int)
        test_predictions = (test_predictions>=0.5).astype(int)
    print("test_predictions:")
    print(test_predictions)
    cm = confusion_matrix(y_test, test_predictions)
    print(cm)
    print(">> resultat de la classification:")
    print(classification_report(y_test, test_predictions))
    TP = cm[0,0]
    TN = cm[1,1]
    FP = cm[0,1]
    FN = cm[1,0]
    classification_accuracy = (TP + TN) / float(TP + TN + FP + FN)
    print('Justesse de la Classification (accuracy) : {0:0.4f}'.format(classification_accuracy))
    classification_error = (FP + FN) / float(TP + TN + FP + FN)
    print('Erreurs de Classification : {0:0.4f}'.format(classification_error))
    recall = TP / float(TP + FN)
    print('Recall ou Sensitivity : {0:0.4f}'.format(recall))
    print("taux de faux positive:")
    false_positive_rate = FP / float(FP + TN)
    print(false_positive_rate)
    print('False Positive Rate : {0:0.4f}'.format(false_positive_rate))
    specificity = TN / (TN + FP)
    print('Specificity : {0:0.4f}'.format(specificity))

In [13]:
grid_search.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_class=11,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [14]:
best_xgb = grid_search.best_estimator_

In [15]:
x_train_3 = x_train.copy()
y_train_3 = y_train.copy()
x_test_3 = x_test.copy()
y_test_3 = y_test.copy()

In [16]:
best_xgb.fit(x_train_3.values,y_train_3)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_class=11,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [17]:
one_stat_result(best_xgb, x_train_3, y_train_3, x_test_3, y_test_3,b_seuil_label=False)

test_predictions:
[10  3  3  2  2  1  2  8  7  6  0  3  3  0  2  1  2  2  3 10  6 10  2  9
  2  8  0  4  3  6  3  6  4  5  2  7  8  3  8  9  9  8  0  4  3  8  0 10
  8  7  5  0  9  4  1  7  2  8 10  8  6  3  8  0  3  9  7  3 10  1  7  4
  9  7  1  3  9  8  1  9  8  9  3  0  3  8  0  3  5  9  2  8  6  8  7  0
  2  0  3  4  8  8  4  4  9  9  2  5  5  8  9  7  7  8  8  3  5  1  2  2
  0  7  2  8  4  6  9  3  0  2  6  1 10  0  6 10  3 10  4  2  6 10]
[[10  1  1  0  0  0  1  0  0  0  0]
 [ 2  1  1  1  2  0  0  3  3  0  0]
 [ 0  1  3  2  0  0  3  0  3  3  1]
 [ 0  0  0  6  1  2  0  0  1  0  0]
 [ 0  1  2  3  2  0  0  0  1  1  0]
 [ 0  0  3  5  1  0  0  0  0  1  1]
 [ 1  2  2  0  0  0  2  1  2  2  2]
 [ 0  1  1  0  0  1  1  7  2  0  1]
 [ 0  1  1  3  1  2  1  0  5  1  2]
 [ 1  0  2  0  2  1  0  0  1  4  1]
 [ 0  0  2  0  1  0  2  0  3  2  2]]
>> resultat de la classification:
              precision    recall  f1-score   support

           0       0.71      0.77      0.74        13
         

In [18]:
## On va essayer de reduire la dimensionalité des données avec une ACP  vu la nature des données, avant de les données aux modèles: 

In [90]:
from sklearn.decomposition import PCA
acp = PCA(n_components=8) # d'apres le pca réalisé dans le fichier preprocessing
x_train_pca = acp.fit_transform(x_train)
x_train_pca
x_test_pca = acp.transform(x_test)
x_test_pca

array([[ 0.05060698,  1.59002884, -0.08955948, ..., -1.02375527,
         0.58219131,  0.77722016],
       [ 1.45759549,  0.18444175,  0.29842092, ..., -0.80829271,
         0.6516778 , -0.49837549],
       [ 3.94532304, -0.57006168,  0.21993827, ...,  0.51929663,
        -0.97645519, -0.69114091],
       ...,
       [ 1.12354428, -0.68442848,  0.24858868, ..., -0.70072378,
         0.35607482, -0.86055038],
       [ 0.82167578, -1.42965125,  1.26132215, ..., -0.176359  ,
        -0.07159885,  0.99064066],
       [-0.49901745,  1.70641893,  0.25321675, ...,  1.23224151,
        -0.07956164,  0.06521126]])

In [91]:
xgb_pca = xgb.XGBClassifier(learning_rate= 0.1,max_depth= 3,
 n_estimators= 100,
 num_class= 11,
 objective= 'multi:softmax',
 reg_lambda= 0.5)

In [92]:
xgb_pca.fit(x_train_pca,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_class=11,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [93]:
one_stat_result(xgb_pca, x_train_pca, y_train_3, x_test_pca, y_test_3,b_seuil_label=False)

test_predictions:
[ 8  8  3 10  4  7  9  6  7  2  0  3  3  0  4  1  8  4  3  5  6  8  8 10
  6  8  0  5  4  6  4  2  6  2  2  0  2  3  6  5 10  9  1  4  3  5  0  9
  8  7  6  8  2  8  2  7  7  4  6  6  6  8  1  1  5  2  6  9  2  1  7  5
  8  7  1  3 10  0  6  6  3 10  3  0  3  8  0  4 10  9  2  6  8  1  7  0
  9  0  3  4 10  5  4  8  9 10 10  5  8  2  2  7  1  8  0  3  4  6  2  3
  0  0  2  8  3  8  9  3  0  3  7  1  9  1  8  8  3  2  4  8  3 10]
[[8 1 0 0 0 0 2 0 2 0 0]
 [2 2 0 0 0 1 1 3 4 0 0]
 [0 0 3 3 1 0 4 0 3 1 1]
 [0 0 0 4 1 2 1 0 1 0 1]
 [0 2 0 2 3 0 1 0 0 1 1]
 [0 0 3 5 2 0 0 0 1 0 0]
 [1 1 1 0 0 0 2 2 3 2 2]
 [2 2 0 0 0 0 3 5 1 0 1]
 [1 1 3 3 2 3 1 0 2 1 0]
 [0 1 2 0 2 1 0 0 2 2 2]
 [0 0 3 1 1 1 0 0 2 2 2]]
>> resultat de la classification:
              precision    recall  f1-score   support

           0       0.57      0.62      0.59        13
           1       0.20      0.15      0.17        13
           2       0.20      0.19      0.19        16
           3       0.2

In [112]:
## pour le model rna:
from keras.models import load_model
from keras.utils import to_categorical
#classifier.load_weights(filepath=path_out+'\best_rna_model_churn.hdf5')
y_train_cat = to_categorical(y_train, num_classes=11)
y_test_cat = to_categorical(y_test, num_classes=11)
# Créer un nouveau modèle avec les mêmes couches et paramètres que le modèle original
best_model = Sequential()
best_model.add(Dense(units=50,kernel_initializer="glorot_uniform",activation='relu',input_dim=x_train.shape[1]))
best_model.add(Dense(units=40,kernel_initializer="glorot_uniform",activation='relu'))
best_model.add(Dense(units=11,kernel_initializer="glorot_uniform",activation='softmax'))
best_model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=["accuracy"])
# Charger les poids du meilleur modèle dans le nouveau modèle
best_model.fit(x_train_, y_train_cat,batch_size=10,epochs=48)


Epoch 1/48
57/57 [==============================] - 1s 2ms/step - loss: 2.3736 - accuracy: 0.1504
Epoch 2/48
57/57 [==============================] - 0s 2ms/step - loss: 2.2202 - accuracy: 0.2584
Epoch 3/48
57/57 [==============================] - 0s 2ms/step - loss: 2.0898 - accuracy: 0.3168
Epoch 4/48
57/57 [==============================] - 0s 2ms/step - loss: 1.9710 - accuracy: 0.3522
Epoch 5/48
57/57 [==============================] - 0s 2ms/step - loss: 1.8853 - accuracy: 0.3540
Epoch 6/48
57/57 [==============================] - 0s 2ms/step - loss: 1.8055 - accuracy: 0.3858
Epoch 7/48
57/57 [==============================] - 0s 2ms/step - loss: 1.7419 - accuracy: 0.3965
Epoch 8/48
57/57 [==============================] - 0s 2ms/step - loss: 1.6636 - accuracy: 0.4265
Epoch 9/48
57/57 [==============================] - 0s 2ms/step - loss: 1.6091 - accuracy: 0.4460
Epoch 10/48
57/57 [==============================] - 0s 2ms/step - loss: 1.5383 - accuracy: 0.4938
Epoch 11/48
57/57 [

In [113]:
from sklearn.metrics import confusion_matrix

# Faire des prédictions avec les données de test
y_pred = best_model.predict(x_test_)

5/5 [==============================] - 0s 3ms/step


In [114]:
# Convertir les prédictions en format catégoriel en utilisant argmax
y_pred_cat = np.argmax(y_pred, axis=1)
# Convertir les étiquettes de classe catégorielles en format continu en utilisant argmax
y_test_cat = np.argmax(y_test_cat, axis=1)

In [115]:
# Calculer la matrice de confusion
cm = confusion_matrix(y_test_cat, y_pred_cat)

print(cm)

[[10  0  2  0  0  0  1  0  0  0  0]
 [ 2  2  1  0  1  1  1  3  2  0  0]
 [ 1  0  4  3  0  0  1  2  2  2  1]
 [ 0  0  0  5  0  2  0  0  2  0  1]
 [ 0  0  2  2  1  0  0  1  1  3  0]
 [ 0  0  2  2  1  3  0  0  1  1  1]
 [ 2  1  2  0  0  2  1  1  1  1  3]
 [ 0  1  2  0  0  0  1  5  3  1  1]
 [ 1  1  2  3  0  0  1  0  4  3  2]
 [ 1  0  1  0  1  1  2  1  1  2  2]
 [ 0  0  2  0  1  0  1  2  4  1  1]]


In [116]:
from sklearn.metrics import precision_score, recall_score, f1_score

def classification_performance(cm):
    # Calculer la précision, le rappel et le score F1 pour chaque classe
    precision = np.diag(cm) / np.sum(cm, axis=0)
    recall = np.diag(cm) / np.sum(cm, axis=1)
    f1 = 2 * precision * recall / (precision + recall)
    
    # Calculer la précision moyenne et le score F1 pondéré
    precision_macro = np.mean(precision)
    f1_weighted = np.sum(f1 * np.sum(cm, axis=1)) / np.sum(cm)
    
    # Afficher les performances de classification pour chaque classe et la précision moyenne et le score F1 pondéré
    for i in range(len(precision)):
        print("Classe {}: Precision = {:.4f}, Recall = {:.4f}, F1 = {:.4f}".format(i, precision[i], recall[i], f1[i]))
    
    print("Precision moyenne: {:.4f}".format(precision_macro))
    print("Score F1 pondéré: {:.4f}".format(f1_weighted))

In [117]:
classification_performance(cm)

Classe 0: Precision = 0.5882, Recall = 0.7692, F1 = 0.6667
Classe 1: Precision = 0.4000, Recall = 0.1538, F1 = 0.2222
Classe 2: Precision = 0.2000, Recall = 0.2500, F1 = 0.2222
Classe 3: Precision = 0.3333, Recall = 0.5000, F1 = 0.4000
Classe 4: Precision = 0.2000, Recall = 0.1000, F1 = 0.1333
Classe 5: Precision = 0.3333, Recall = 0.2727, F1 = 0.3000
Classe 6: Precision = 0.1111, Recall = 0.0714, F1 = 0.0870
Classe 7: Precision = 0.3333, Recall = 0.3571, F1 = 0.3448
Classe 8: Precision = 0.1905, Recall = 0.2353, F1 = 0.2105
Classe 9: Precision = 0.1429, Recall = 0.1667, F1 = 0.1538
Classe 10: Precision = 0.0833, Recall = 0.0833, F1 = 0.0833
Precision moyenne: 0.2651
Score F1 pondéré: 0.2550
